### Вычисление рейтингов игроков Dota 2 с помощью TrueSkill

In [3]:
import numpy as np
import pandas as pd

import trueskill
import pprint

In [2]:
match_outcomes = pd.read_csv('/data/notebook_files/match_outcomes.csv')

match_outcomes.head()

,match_id,account_id_0,account_id_1,account_id_2,account_id_3,account_id_4,start_time,parser_version,win,rad
0,1636204962,34549,0,0,-51743434,-120875154,1437014585,12,1,0
1,1636204962,0,61598,138825,0,207232,1437014585,12,0,1
2,1636322679,0,-44943233,-240360907,19599,0,1437019968,12,0,0
3,1636322679,-97530201,0,0,0,-116349387,1437019968,12,1,1
4,1637385965,0,0,0,104738,0,1437052551,12,1,0


In [5]:
ts = trueskill.TrueSkill(draw_probability=0)
ts

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=0.0%)

In [4]:

unique_acc_ids = pd.DataFrame(match_outcomes.iloc[:,1:6].unstack().unique(),
                              columns=['account_id'])
print('Number of unique account ids:', unique_acc_ids.shape[0],'\n')
print(unique_acc_ids.head())

Number of unique account ids: 834226 

   account_id
0       34549
1           0
2   -97530201
3  -123447796
4  -108454938


In [5]:

rating_dict = dict()
for i in unique_acc_ids.values.ravel():
    rating_dict[i] = ts.create_rating()

In [6]:
len(rating_dict.keys())

834226

In [7]:

pprint.pprint(str(rating_dict)[:253]+'}')

('{34549: trueskill.Rating(mu=25.000, sigma=8.333), 0: '
 'trueskill.Rating(mu=25.000, sigma=8.333), -97530201: '
 'trueskill.Rating(mu=25.000, sigma=8.333), -123447796: '
 'trueskill.Rating(mu=25.000, sigma=8.333), -108454938: '
 'trueskill.Rating(mu=25.000, sigma=8.333}')


In [6]:
def replace_anonymous(team_ids, rating_dict, team_name, ts_obj):   
    team_dict = dict()
    non_anon = []
    
    sum_mu = 0
    sum_sigma = 0
    
    non_anons = 0
    for i in team_ids:
        if i != 0 and i in rating_dict.keys():
            sum_mu += rating_dict[i].mu
            sum_sigma += rating_dict[i].sigma
            non_anons += 1
    

    if non_anons == 0:
        for e in range(5):
            team_dict[team_name + str(e)] = ts_obj.create_rating()
        return team_dict
        
    mean_mu = sum_mu/non_anons
    mean_sigma = sum_sigma/non_anons

    for e,i in enumerate(team_ids):
        
        if i == 0 or i not in rating_dict.keys():
            team_dict[team_name + str(e)] = ts_obj.create_rating(mean_mu, mean_sigma)
        else:
            team_dict[i] = rating_dict[i]
        
    return team_dict

def update_ratings(new_ratings, rating_dict):
    for key in new_ratings.keys():
        if type(key) is not str:
            rating_dict[key] = new_ratings[key]
    return rating_dict

In [9]:
match_groups = match_outcomes.groupby('match_id')

# just to see what this looks like
for e,group in enumerate(match_groups):
    break
print('The group key(match_id):',group[0],'\n')
print('The group data with dire on top row and radiant on bottom.\n',group[1])

The group key(match_id): 1636204962 

The group data with dire on top row and radiant on bottom.
      match_id  account_id_0  account_id_1  account_id_2  account_id_3  \
0  1636204962         34549             0             0     -51743434   
1  1636204962             0         61598        138825             0   

   account_id_4  start_time  parser_version  win  rad  
0    -120875154  1437014585              12    1    0  
1        207232  1437014585              12    0    1  


In [10]:
for e,group in enumerate(match_groups):
    radiant_ids = group[1].iloc[1,1:6] 
    dire_ids = group[1].iloc[0,1:6]
    
    radiant_dict = replace_anonymous(radiant_ids, rating_dict, 'radiant', ts)
    dire_dict = replace_anonymous(dire_ids, rating_dict, 'dire', ts)
    
    if group[1].iloc[1,8] == 1:
        updated_radiant, updated_dire = ts.rate([radiant_dict, dire_dict], ranks=[0,1]) # 0 - первое место, 1 - второе место
    else:
        updated_radiant, updated_dire = ts.rate([radiant_dict, dire_dict], ranks=[1,0])
    
    rating_dict = update_ratings(updated_radiant, rating_dict)
    rating_dict = update_ratings(updated_dire, rating_dict)

In [11]:
rating_arr = np.zeros((len(rating_dict.keys()), 3))

for e,i in enumerate(rating_dict.keys()):
    rating_arr[e,0] = i
    rating_arr[e,1] = rating_dict[i].mu
    rating_arr[e,2] = rating_dict[i].sigma

In [15]:
rating_df = pd.DataFrame(rating_arr, columns=['account_id', 'trueskill_mu','trueskill_sigma'])
rating_df.head(10)

,account_id,trueskill_mu,trueskill_sigma
0,34549.0,25.496363,5.277149
1,0.0,25.000000,8.333333
2,-97530201.0,26.880745,8.118755
3,-123447796.0,24.867150,3.543024
4,-108454938.0,32.028790,3.659563
5,320093.0,26.680175,5.869468
6,-142035.0,24.473918,4.942341
7,179628.0,29.610686,5.286551
8,-167470448.0,22.376919,7.685111
9,103832.0,23.776348,6.962687


In [16]:
filtered_rating_df = rating_df.loc[rating_df['account_id'] > 0]
filtered_rating_df

,account_id,trueskill_mu,trueskill_sigma
0,34549.0,25.496363,5.277149
5,320093.0,26.680175,5.869468
7,179628.0,29.610686,5.286551
9,103832.0,23.776348,6.962687
13,31123.0,31.485674,3.400521
...,...,...,...
834210,36698.0,22.925483,8.087499
834215,122332.0,24.645972,8.216979
834216,14572.0,27.460018,7.984721
834223,156640.0,26.614940,8.075445


In [ ]:
#filtered_rating_df.to_csv("/data/notebook_files/trueskill_ratings.csv")

In [7]:
import itertools
import math

def win_probability(team1, team2, ts):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (trueskill.BETA * trueskill.BETA) + sum_sigma)
    return ts.cdf(delta_mu / denom)

In [8]:
filtered_rating_df = pd.read_csv("/data/notebook_files/trueskill_ratings.csv")

In [9]:
rating_dict = dict()
filtered_rating_df = filtered_rating_df.reset_index()  # make sure indexes pair with number of rows
for index, row in filtered_rating_df.iterrows():
    rating_dict[row.account_id] = ts.create_rating(mu=row.trueskill_mu, sigma=row.trueskill_sigma)

rating_dict

{34549.0: trueskill.Rating(mu=25.496, sigma=5.277),
 320093.0: trueskill.Rating(mu=26.680, sigma=5.869),
 179628.0: trueskill.Rating(mu=29.611, sigma=5.287),
 103832.0: trueskill.Rating(mu=23.776, sigma=6.963),
 31123.0: trueskill.Rating(mu=31.486, sigma=3.401),
 245558.0: trueskill.Rating(mu=29.924, sigma=4.107),
 111635.0: trueskill.Rating(mu=25.118, sigma=4.868),
 161203.0: trueskill.Rating(mu=28.927, sigma=7.839),
 183021.0: trueskill.Rating(mu=21.437, sigma=6.582),
 205333.0: trueskill.Rating(mu=29.491, sigma=5.672),
 26336.0: trueskill.Rating(mu=26.098, sigma=3.920),
 42419.0: trueskill.Rating(mu=28.690, sigma=2.563),
 61083.0: trueskill.Rating(mu=24.736, sigma=1.803),
 116094.0: trueskill.Rating(mu=28.606, sigma=6.750),
 293752.0: trueskill.Rating(mu=26.030, sigma=5.309),
 192607.0: trueskill.Rating(mu=30.635, sigma=4.272),
 62837.0: trueskill.Rating(mu=28.613, sigma=4.546),
 1778.0: trueskill.Rating(mu=28.522, sigma=5.992),
 2859.0: trueskill.Rating(mu=26.852, sigma=3.446),
 92

In [10]:
players = pd.read_csv('/data/notebook_files/players.csv')
match = pd.read_csv('/data/notebook_files/match.csv')

In [11]:
# join ratings with the players table
match_data = pd.merge(players, filtered_rating_df, how='left', left_on='account_id', right_on='account_id')
match_data = pd.merge(match_data, match, how='left', left_on='match_id', right_on='match_id')
#frequent_players =  players.query('total_matches >= 50').copy()

# some of these variables don't have order, and I don't want to plot them against skill
#to_drop = ['match_id', 'hero_id','player_slot', 
#           'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 
#           'leaver_status', 'unit_order_none', 'stuns','total_matches',
#           'total_wins', 'trueskill_sigma']

#frequent_players.drop(to_drop, axis=1, inplace=True)

#frequent_players.shape

columns = ["match_id", "account_id", "player_slot", "radiant_win", "trueskill_mu", "trueskill_sigma"]

match_data = match_data[columns]
match_data = match_data.sort_values(by="match_id", ascending=True)
match_data.head(100)

,match_id,account_id,player_slot,radiant_win,trueskill_mu,trueskill_sigma
0,0,0,0,True,NaN,NaN
1,0,1,1,True,26.232906,4.854238
2,0,0,2,True,NaN,NaN
3,0,2,3,True,27.614505,6.550771
4,0,3,4,True,20.221006,5.961434
...,...,...,...,...,...,...
93,9,0,3,False,NaN,NaN
92,9,0,2,False,NaN,NaN
91,9,0,1,False,NaN,NaN
90,9,0,0,False,NaN,NaN


In [12]:
match_groups = match_data.groupby('match_id')

# just to see what this looks like
for e,group in enumerate(match_groups):
    break

players = group[1].sort_values(by='player_slot', ascending=True)
radiant_ids = players.iloc[0:5,1]
dire_ids = players.iloc[5:10,1]

radiant_dict = replace_anonymous(radiant_ids, rating_dict, 'radiant', ts)
dire_dict = replace_anonymous(dire_ids, rating_dict, 'dire', ts)

radiant_mu = 0
radiant_sigma = 0
dire_mu = 0
dire_sigma = 0

for key in radiant_dict:
    radiant_mu += (radiant_dict[key].mu / len(radiant_dict))
    radiant_sigma += (radiant_dict[key].sigma / len(radiant_dict))
for key in dire_dict:
    dire_mu += (dire_dict[key].mu / len(dire_dict))
    dire_sigma += (dire_dict[key].sigma / len(dire_dict))

radiant_win_predict = win_probability(radiant_dict.values(), dire_dict.values(), ts)

correct_predict = False
efficiency = 0
if (radiant_win_predict >= 0.5 and players.iloc[0,3] == True) or (radiant_win_predict <= 0.5 and players.iloc[0,3] == False):
    correct_predict = True
    efficiency = abs(radiant_win_predict - 0.5)
else:
    efficiency = -abs(radiant_win_predict - 0.5)

new_row = pd.DataFrame(data={"match_id": players.iloc[0,0], "radiant_mu": radiant_mu, "radiant_sigma": radiant_sigma, "dire_mu": dire_mu, "dire_sigma": dire_sigma, "radiant_win": players.iloc[0,3], "correct_predict": correct_predict, "radiant_win_predict": radiant_win_predict, "efficiency": efficiency}, index=[0])
#results_prediction = pd.concat([results_prediction, new_row], ignore_index=True)
new_row

,match_id,radiant_mu,radiant_sigma,dire_mu,dire_sigma,radiant_win,correct_predict,radiant_win_predict,efficiency
0,0,24.689472,5.788814,31.246125,4.680773,True,False,0.062543,-0.437457


In [13]:
results_prediction = pd.DataFrame(columns=["match_id", "radiant_mu", "radiant_sigma", "dire_mu", "dire_sigma", "radiant_win", "correct_predict", "radiant_win_predict", "efficiency"])

In [14]:
for e,group in enumerate(match_groups):
    players = group[1].sort_values(by='player_slot', ascending=True)
    radiant_ids = players.iloc[0:5,1]
    dire_ids = players.iloc[5:10,1]

    radiant_dict = replace_anonymous(radiant_ids, rating_dict, 'radiant', ts)
    dire_dict = replace_anonymous(dire_ids, rating_dict, 'dire', ts)

    radiant_mu = 0
    radiant_sigma = 0
    dire_mu = 0
    dire_sigma = 0

    for key in radiant_dict:
        radiant_mu += (radiant_dict[key].mu / len(radiant_dict))
        radiant_sigma += (radiant_dict[key].sigma / len(radiant_dict))
    for key in dire_dict:
        dire_mu += (dire_dict[key].mu / len(dire_dict))
        dire_sigma += (dire_dict[key].sigma / len(dire_dict))

    radiant_win_predict = win_probability(radiant_dict.values(), dire_dict.values(), ts)

    correct_predict = False
    efficiency = 0
    if (radiant_win_predict >= 0.5 and players.iloc[0,3] == True) or (radiant_win_predict <= 0.5 and players.iloc[0,3] == False):
        correct_predict = True
        efficiency = abs(radiant_win_predict - 0.5)
    else:
        efficiency = -abs(radiant_win_predict - 0.5)

    new_row = pd.DataFrame(data={"match_id": players.iloc[0,0], "radiant_mu": radiant_mu, "radiant_sigma": radiant_sigma, "dire_mu": dire_mu, "dire_sigma": dire_sigma, "radiant_win": players.iloc[0,3], "correct_predict": correct_predict, "radiant_win_predict": radiant_win_predict, "efficiency": efficiency}, index=[0])
    results_prediction = pd.concat([results_prediction, new_row], ignore_index=True)

    if group[1].iloc[0,3]:
        updated_radiant, updated_dire = ts.rate([radiant_dict, dire_dict], ranks=[0,1]) # 0 - первое место, 1 - последнее место
    else:
        updated_radiant, updated_dire = ts.rate([radiant_dict, dire_dict], ranks=[1,0])

    rating_dict = update_ratings(updated_radiant, rating_dict)
    rating_dict = update_ratings(updated_dire, rating_dict)

In [15]:
results_prediction.head(100)

,match_id,radiant_mu,radiant_sigma,dire_mu,dire_sigma,radiant_win,correct_predict,radiant_win_predict,efficiency
0,0,24.689472,5.788814,31.246125,4.680773,True,False,0.062543,-0.437457
1,1,23.911222,4.828813,29.321903,5.199332,False,True,0.097672,0.402328
2,2,28.470775,7.955096,25.000000,8.333333,False,False,0.725655,-0.225655
3,3,21.141360,7.506343,24.479283,6.180934,False,True,0.256609,0.243391
4,4,27.535748,5.681794,28.210129,5.970909,True,False,0.441727,-0.058273
...,...,...,...,...,...,...,...,...,...
95,95,26.355108,2.823191,28.073260,3.694767,False,True,0.309163,0.190837
96,96,26.056251,7.317868,21.751250,7.942223,True,True,0.783025,0.283025
97,97,22.441421,6.757101,25.000000,8.333333,True,False,0.320485,-0.179515
98,98,24.287694,6.130537,24.124062,7.236983,False,False,0.512986,-0.012986


In [16]:
(results_prediction["efficiency"].sum() / len(results_prediction.index)) * 1000

7.606695908870581

In [17]:
count = results_prediction.groupby(by="correct_predict", as_index=True)["match_id"].count()
count

In [18]:
# Процент правильных предсказаний
count.iloc[1] / (count.iloc[1] + count.iloc[0])

0.51702

In [19]:
results_prediction.to_csv("/data/notebook_files/trueskill_results_prediction_filled.csv")

In [20]:
# transform the updated rating dictionary into a pandas DataFrame
rating_arr = np.zeros((len(rating_dict.keys()), 3))

for e,i in enumerate(rating_dict.keys()):
    rating_arr[e,0] = i
    rating_arr[e,1] = rating_dict[i].mu
    rating_arr[e,2] = rating_dict[i].sigma

In [21]:
rating_df = pd.DataFrame(rating_arr, columns=['account_id', 'trueskill_mu','trueskill_sigma'])
rating_df.head(10)

,account_id,trueskill_mu,trueskill_sigma
0,34549.0,21.518613,4.749016
1,320093.0,26.680175,5.869468
2,179628.0,29.610686,5.286551
3,103832.0,25.637964,6.756268
4,31123.0,30.397861,3.351353
5,245558.0,29.923902,4.106992
6,111635.0,22.646814,4.641562
7,161203.0,28.927350,7.838732
8,183021.0,21.436980,6.581934
9,205333.0,29.490660,5.671873


In [22]:
#rating_df.to_csv("/data/notebook_files/trueskill_ratings_950k.csv")

In [23]:
filtered_rating_df = pd.read_csv("/data/notebook_files/trueskill_ratings.csv")

In [24]:
rating_dict = dict()
filtered_rating_df = filtered_rating_df.reset_index()  # make sure indexes pair with number of rows
for index, row in filtered_rating_df.iterrows():
    rating_dict[row.account_id] = ts.create_rating(mu=row.trueskill_mu, sigma=row.trueskill_sigma)

rating_dict

{34549.0: trueskill.Rating(mu=25.496, sigma=5.277),
 320093.0: trueskill.Rating(mu=26.680, sigma=5.869),
 179628.0: trueskill.Rating(mu=29.611, sigma=5.287),
 103832.0: trueskill.Rating(mu=23.776, sigma=6.963),
 31123.0: trueskill.Rating(mu=31.486, sigma=3.401),
 245558.0: trueskill.Rating(mu=29.924, sigma=4.107),
 111635.0: trueskill.Rating(mu=25.118, sigma=4.868),
 161203.0: trueskill.Rating(mu=28.927, sigma=7.839),
 183021.0: trueskill.Rating(mu=21.437, sigma=6.582),
 205333.0: trueskill.Rating(mu=29.491, sigma=5.672),
 26336.0: trueskill.Rating(mu=26.098, sigma=3.920),
 42419.0: trueskill.Rating(mu=28.690, sigma=2.563),
 61083.0: trueskill.Rating(mu=24.736, sigma=1.803),
 116094.0: trueskill.Rating(mu=28.606, sigma=6.750),
 293752.0: trueskill.Rating(mu=26.030, sigma=5.309),
 192607.0: trueskill.Rating(mu=30.635, sigma=4.272),
 62837.0: trueskill.Rating(mu=28.613, sigma=4.546),
 1778.0: trueskill.Rating(mu=28.522, sigma=5.992),
 2859.0: trueskill.Rating(mu=26.852, sigma=3.446),
 92

In [25]:
players = pd.read_csv('/data/notebook_files/players.csv')
match = pd.read_csv('/data/notebook_files/match.csv')

In [26]:
# join ratings with the players table
match_data = pd.merge(players, filtered_rating_df, how='left', left_on='account_id', right_on='account_id')
match_data = pd.merge(match_data, match, how='left', left_on='match_id', right_on='match_id')
#frequent_players =  players.query('total_matches >= 50').copy()

# some of these variables don't have order, and I don't want to plot them against skill
#to_drop = ['match_id', 'hero_id','player_slot', 
#           'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 
#           'leaver_status', 'unit_order_none', 'stuns','total_matches',
#           'total_wins', 'trueskill_sigma']

#frequent_players.drop(to_drop, axis=1, inplace=True)

#frequent_players.shape

columns = ["match_id", "account_id", "player_slot", "radiant_win", "trueskill_mu", "trueskill_sigma"]

match_data = match_data[columns]
match_data.dropna(inplace=True)
match_data = match_data.sort_values(by="match_id", ascending=True)
match_data.head(100)

,match_id,account_id,player_slot,radiant_win,trueskill_mu,trueskill_sigma
1,0,1,1,True,26.232906,4.854238
3,0,2,3,True,27.614505,6.550771
4,0,3,4,True,20.221006,5.961434
5,0,4,128,True,26.773302,5.322094
7,0,5,130,True,32.190551,2.937140
...,...,...,...,...,...,...
183,18,98,3,False,23.778578,4.098880
184,18,99,4,False,31.598454,3.786333
185,18,100,128,False,30.666680,2.547452
187,18,101,130,False,27.902340,3.672764


In [27]:
match_groups = match_data.groupby('match_id')

In [28]:
results_prediction = pd.DataFrame(columns=["match_id", "radiant_rating", "radiant_rd", "dire_rating", "dire_rd", "radiant_win", "correct_predict", "radiant_win_predict", "efficiency"])

In [29]:
for e,group in enumerate(match_groups):
    if len(group[1].index) != 10:
        continue

    players = group[1].sort_values(by='player_slot', ascending=True)
    radiant_ids = players.iloc[0:5,1]
    dire_ids = players.iloc[5:10,1]

    radiant_dict = replace_anonymous(radiant_ids, rating_dict, 'radiant', ts)
    dire_dict = replace_anonymous(dire_ids, rating_dict, 'dire', ts)

    radiant_mu = 0
    radiant_sigma = 0
    dire_mu = 0
    dire_sigma = 0

    for key in radiant_dict:
        radiant_mu += (radiant_dict[key].mu / len(radiant_dict))
        radiant_sigma += (radiant_dict[key].sigma / len(radiant_dict))
    for key in dire_dict:
        dire_mu += (dire_dict[key].mu / len(dire_dict))
        dire_sigma += (dire_dict[key].sigma / len(dire_dict))

    radiant_win_predict = win_probability(radiant_dict.values(), dire_dict.values(), ts)

    correct_predict = False
    efficiency = 0
    if (radiant_win_predict >= 0.5 and players.iloc[0,3] == True) or (radiant_win_predict <= 0.5 and players.iloc[0,3] == False):
        correct_predict = True
        efficiency = abs(radiant_win_predict - 0.5)
    else:
        efficiency = -abs(radiant_win_predict - 0.5)

    new_row = pd.DataFrame(data={"match_id": players.iloc[0,0], "radiant_mu": radiant_mu, "radiant_sigma": radiant_sigma, "dire_mu": dire_mu, "dire_sigma": dire_sigma, "radiant_win": players.iloc[0,3], "correct_predict": correct_predict, "radiant_win_predict": radiant_win_predict, "efficiency": efficiency}, index=[0])
    results_prediction = pd.concat([results_prediction, new_row], ignore_index=True)

    if group[1].iloc[0,3]: # radiant won
        updated_radiant, updated_dire = ts.rate([radiant_dict, dire_dict], ranks=[0,1]) # for ranks 0 is winner 
    else: # dire won
        updated_radiant, updated_dire = ts.rate([radiant_dict, dire_dict], ranks=[1,0])

    rating_dict = update_ratings(updated_radiant, rating_dict)
    rating_dict = update_ratings(updated_dire, rating_dict)

In [30]:
results_prediction

,match_id,radiant_rating,radiant_rd,dire_rating,dire_rd,radiant_win,correct_predict,radiant_win_predict,efficiency,radiant_mu,radiant_sigma,dire_mu,dire_sigma
0,110,NaN,NaN,NaN,NaN,False,False,0.514956,-0.014956,29.712382,3.262200,29.583416,3.424133
1,114,NaN,NaN,NaN,NaN,True,False,0.486738,-0.013262,23.977857,2.610924,24.085628,3.217254
2,115,NaN,NaN,NaN,NaN,True,False,0.177505,-0.322495,23.930627,5.335546,28.236544,6.433854
3,164,NaN,NaN,NaN,NaN,False,True,0.295927,0.204073,25.626378,6.490867,27.971889,3.999721
4,183,NaN,NaN,NaN,NaN,True,True,0.897590,0.397590,28.355820,4.600308,22.830594,5.993174
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,49743,NaN,NaN,NaN,NaN,False,False,0.829819,-0.329819,27.063165,2.612966,23.313839,5.794558
2104,49767,NaN,NaN,NaN,NaN,False,False,0.567206,-0.067206,26.054555,5.573056,25.315232,5.071353
2105,49769,NaN,NaN,NaN,NaN,True,True,0.731116,0.231116,26.632706,4.049543,24.164262,4.504512
2106,49808,NaN,NaN,NaN,NaN,True,True,0.706042,0.206042,28.226607,4.021034,26.051773,5.078753


In [31]:
(results_prediction["efficiency"].sum()/ len(results_prediction.index)) * 1000

15.871528160809307

In [32]:
count = results_prediction.groupby(by="correct_predict", as_index=True)["match_id"].count()
count

In [33]:
# Процент правильных предсказаний
count.iloc[1] / (count.iloc[1] + count.iloc[0])

0.540796963946869

In [34]:
results_prediction.to_csv("/data/notebook_files/trueskill_results_prediction_full_only.csv")